In [18]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name

def remove_accents(text):
    return unidecode(text)


def get_country_code(country):
    matching_rows = countries_codes.loc[countries_codes['Country'] == country, ' Code']
    return matching_rows.iloc[0] if not matching_rows.empty else None

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

from fuzzywuzzy import process

#PLAYER DATA - MARKET VALUE AND SALARY
leagues_salary = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1//CSVs we use often/latest_capology_data_money_fixed.csv')

leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/MarketValuesComplete.csv')


leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')

#COUNTRY CODES
countries_codes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/countries_and_codes.csv')

#FIFA RANKINGS
fifa_rankings = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/fifa_ranking-2023-10-26 - fifa_ranking-2023-07-20.csv')

#MATCH DATA - LINEUPS AND RESULTS
# lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_march_24.csv')
lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_april_29.csv')
results_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_results_data_march_24.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_25874/1460689172.py:64: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')


In [2]:
leagues_value['Accents Removed'] = leagues_value['Name'].apply(lambda x: unidecode(x))
leagues_value['Team 1 Code'] = leagues_value['Nationality'].apply(get_country_code)

In [107]:
combined_df = pd.read_csv('combined_df_may_7_newest.csv')
for index, row in combined_df.iterrows():
    if row['Status'] == 'SUCCESS' and row['Match Case'] == 'none':
        combined_df.at[index, 'Match Case'] = 'single'

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_25874/2939037280.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv('combined_df_may_7_newest.csv')


In [108]:
combined_df[~combined_df['Competition'].str.startswith('U-20')]

,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
0,0,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Anthony Moris,400000.0,False,working,False,Anthony Moris,A Moris,single
1,1,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Laurent Jans,1200000.0,False,working,False,Laurent Jans,L Jans,single
2,2,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Kevin Malget,150000.0,False,working,False,Kevin Malget,K Malget,single
3,3,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Mario Mutsch,125000.0,False,working,False,Mario Mutsch,M Mutsch,single
4,4,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Chris Philipps,1250000.0,False,working,False,Chris Philipps,C Philipps,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89459,89459,1381,"Olympics, 2020",Brazil vs Ivory Coast,2021-07-25,Brazil,BR,2021,SUCCESS,"['Paulinho', 'Marquinhos', 'Fabinho']",11000000.0,False,Found in Curr Season,False,Claudinho,Claudinho,single
89460,89460,1383,"Olympics, 2020",Saudi Arabia vs Brazil,2021-07-28,Brazil,BR,2021,SUCCESS,"['Paulinho', 'Marquinhos', 'Fabinho']",11000000.0,False,Found in Curr Season,False,Claudinho,Claudinho,single
89461,89461,1387,"Olympics, 2020",Brazil vs Egypt,2021-07-31,Brazil,BR,2021,SUCCESS,"['Paulinho', 'Marquinhos', 'Fabinho']",11000000.0,False,Found in Curr Season,False,Claudinho,Claudinho,single
89462,89462,1389,"Olympics, 2020",Mexico vs Brazil,2021-08-03,Brazil,BR,2021,SUCCESS,"['Paulinho', 'Marquinhos', 'Fabinho']",11000000.0,False,Found in Curr Season,False,Claudinho,Claudinho,single


In [245]:
def add_correct_money(money_column):
    if money_column == 0:
        return 0

    if 'm' in money_column:
        value_rough = money_column.split('m')[0].split('€')[1]
        millinum = (float(value_rough) * 1000000)
        return millinum
    elif 'k' in money_column:
        #print(i, 'k')
        value_rough = money_column.split('k')[0].split('€')[1]
        millinum = (float(value_rough) * 1000)

        return millinum
    elif '-' in money_column:
        return 0
    else:
        return 0


def in_season_around(player_id, curr_season, age):
    if age == "Not Listed": age = 30
    age = int(age)
    year_before = int(curr_season) -1
    year_after = int(curr_season) + 1
    before_season = leagues_value[(leagues_value['PlayerID'] == player_id) & (leagues_value['Season'] == year_before)] #leagues_value_large #leagues_value
    after_season =  leagues_value[(leagues_value['PlayerID'] == player_id) & (leagues_value['Season'] == year_after)]
    if before_season.empty and after_season.empty:
        return -1, "Both Empty"
    if not before_season.empty and after_season.empty:
        if before_season.iloc[0]['Market Value'] != "-":
            if before_season.iloc[0]['Market Value'] == 0:
                return 0, "Only Before + Zero Before"
            else:
                if age >= 30:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * .8), "Only Before" #add_correct_money
                else:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * 1.2), "Only Before" #add_correct_money
        else:
            return 0, "Only Before X"
    if not after_season.empty and before_season.empty:
        
        if after_season.iloc[0]['Market Value'] != "-":
            if after_season.iloc[0]['Market Value'] == 0:
                return 0, "Only After + Zero After"
            else:
                if age <= 30:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * .8), "Only After"
                else:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * 1.2), "Only After"
        else:
            return 0, "Only After X"
    if not before_season.empty and not after_season.empty:
        #print('hi')
        before_season_val = before_season.iloc[0]['Market Value']
        after_season_val = after_season.iloc[0]['Market Value']
        #print(before_season_val, after_season_val)
        if before_season_val != "-" and after_season_val != "-":
            # if type(before_season_val) == str:
            #     before_season_val = add_correct_money(before_season_val)
            # if type(after_season_val) == str:
            #     after_season_val = add_correct_money(after_season_val)
            return ((float(add_correct_money(before_season.iloc[0]['Market Value'])) + float(add_correct_money(after_season.iloc[0]['Market Value'])) / 2)), "Used Average of Season Before and After"
        elif before_season_val != "-" and after_season_val == "-":
            if age >= 30:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * .8)), "Used Season Before * .8 But Had Both"
            else:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * 1.2)), "Used Season Before * 1.2 But Had Both"
        elif before_season_val == "-" and after_season_val != "-":
            if age <= 30:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * .8)), "Used Season After * .8 But Had Both"
            else:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * 1.2)), "Used Season After * 1.2 But Had Both"
        else:
            return 0, "Had Both but Both Were Empty"

def find_in_transfermarkt(player_name, season, country_code, database_name):
    #print(player_name, country_code, season)
    row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == season) & (database_name['Team 1 Code'] == country_code)]
    one_up_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)+1) & (database_name['Team 1 Code'] == country_code)]
    one_down_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)-1) & (database_name['Team 1 Code'] == country_code)]
    print(len(row), len(one_down_row), len(one_up_row))
    final_value = 0
    if not row.empty:
        #print('yeeea')
        player_id = row.iloc[0]['PlayerID']
        age = row.iloc[0]['Age']
        marketval = row.iloc[0]['Market Value']
        if type(marketval) == str:
            #print('der')
            if marketval == '-':
                marketval = 0
            elif '€' in marketval:
                marketval = add_correct_money(marketval)
            marketval = float((marketval)) #add_correct_money
        if marketval != 0:
            #print('wooo')
            #print('***', player_name, marketval)
            return marketval, "Found in Curr Season"
        else:
            #print(marketval)
            new_mv, reason = in_season_around(player_id, season, age)
            print(player_name, new_mv, reason)
            if new_mv == -1 or new_mv == 0 or new_mv == "-":
                return 0, reason
            else:
                return new_mv, reason
    elif not one_up_row.empty:
        player_id = one_up_row.iloc[0]['PlayerID']
        age = one_up_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    elif not one_down_row.empty:
        player_id = one_down_row.iloc[0]['PlayerID']
        age = one_down_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    else:
        return 0, "Did Not Find in Curr Season, One Up, or One Down"
    
def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''

def vet_group_using_initials(jersey, names_found, names_same_nation_last_name):
    correct_names_list = []

    jersey_tokens_initials = [token[0] for token in re.split(r'\s+|[-–]', jersey)]

    names_found_tokens_initials = [token[0] for token in re.split(r'\s+|[-–]', names_found)]

    for name in names_same_nation_last_name:
        name_initials = [token[0] for token in re.split(r'\s+|[-–]', name)]
        
        if set(jersey_tokens_initials) == set(name_initials):
            correct_names_list.append(name)
        elif set(jersey_tokens_initials).issubset(set(name_initials)):
            correct_names_list.append(name)
        #else:
            #print('fail')

    if set(jersey_tokens_initials) == set(names_found_tokens_initials):
        correct_names_list.append(names_found)
    elif set(jersey_tokens_initials).issubset(set(names_found_tokens_initials)):
            correct_names_list.append(names_found)


    return correct_names_list


from fuzzywuzzy import process

def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}

## Name Confusion

In [60]:
name_confusion = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Red Groups/name_confusion.csv')
name_confusion = name_confusion.drop(columns={'Unnamed: 0.1', 'Unnamed: 0'})


In [134]:
done_cases = 0
correct_cases = 0
not_working_cases = 0
for index, row in name_confusion.iterrows():
    if row['Status New'] != 'Done':

        match = row['Match']
        date = row['Date']
        jersey = row['ORIGINAL JERSEY']
        if jersey == 'Hosseini':
            name_confusion.at[index, 'Name(s) Found'] = 'Majid Hosseini'
            name_confusion.at[index, 'ORIGINAL JERSEY'] = 'Majid Hosseini'
            jersey = 'Majid Hosseini'
        
        full_num = row['Season']
        country_code_this = row['Country Code']

        name_confusion_matching_rows = name_confusion[(name_confusion['Match'] == match) & (name_confusion['Date'] == date) & (name_confusion['ORIGINAL JERSEY'] == jersey)]

        #search within dataset
        if len(name_confusion_matching_rows) >= 2:
            print(index, match, date, jersey, row['Name(s) Found'])
            name_confusion.at[index, 'Status New'] = 'Done'
        else:
            not_working_cases += 1
            #Search within the combined DF, in case there's another row with one of the names 
            if 'john' != 'john':
                list_of_names = eval(row['Name(s) Found'])
                for name in list_of_names:
                    name_confusion_matching_rows_big_df = combined_df[(combined_df['Match'] == match) & (combined_df['Date'] == date) & (combined_df['Name(s) Found'] == name)]
                    if len(name_confusion_matching_rows_big_df) >= 1:
                        list_of_names.remove(name)
                        if len(list_of_names) == 1:
                            
                            correct_name = list_of_names[0]
                            print(f"{index}. original was {row['Name(s) Found']} now it's {correct_name}. found {name}. jersey says {jersey}")
                                
                            number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)

                            if correct_name == 'Mohamed Zaabia':
                                print('yeh')
                                number = 0
                                reason = 'Had both but both were 0'
                            #print(player_this, reason, add_correct_money(number))
                            #add a new row to lookup dataset 1 
                            if type(number) == float:
                                0==0
                            else:
                                number = add_correct_money(number)
                            
                            if reason != "Did Not Find in Curr Season, One Up, or One Down":
                                print('**')
                                print(index, jersey, correct_name, number, reason)
                            #if reason == 'Found in Curr Season':
                                name_confusion.at[index, 'Name(s) Found'] = correct_name
                                name_confusion.at[index, 'Status New'] = 'Done'
                                name_confusion.at[index, 'Market Value New'] = number
                                name_confusion.at[index, 'Match Case'] = 'single'
                                correct_cases += 1
                            else:
                                print('not working', index, number, reason, correct_name, full_num)
                                
                        else:
                            print(index, 'wtf')
                        #print(index, row['Name(s) Found'], jersey, match, date, name)
                    else:
                        0==0
            else:
                
                if jersey == 'Majid Hosseini':
                    list_of_names = 'Majid Hosseini'
                elif jersey == 'Al-Dawsari':
                    list_of_names = 'Salem Al-Dawsari'
                else:
                    list_of_names = eval(row['Name(s) Found'])
                #print('hey', index, match, date, jersey, list_of_names)
                if type(list_of_names) == str:
                    0==0
                    print(list_of_names)
                    correct_name = list_of_names
                    number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                    if type(number) == float:
                        0==0
                    else:
                        number = add_correct_money(number)
                    
                    if reason != "Did Not Find in Curr Season, One Up, or One Down":
                        print('**')
                        print(index, jersey, correct_name, number, reason)
                    #if reason == 'Found in Curr Season':
                        name_confusion.at[index, 'Name(s) Found'] = correct_name
                        name_confusion.at[index, 'Status New'] = 'Done'
                        name_confusion.at[index, 'Market Value New'] = number
                        name_confusion.at[index, 'Match Case'] = 'single'
                        correct_cases += 1
                    else:
                        print('not working', index, number, reason, correct_name, full_num)

                    #print('holy shit', combined_df[combined_df['ORIGINAL JERSEY'] == jersey]['Name(s) Found'].unique())
                else:
                    #if there's only one name in the dataset, you can assume it's that name
                    # existing_data_names_found_this_jersey = combined_df[combined_df['ORIGINAL JERSEY'] == jersey]['Name(s) Found'].unique()
                    # if len(existing_data_names_found_this_jersey) == 1:
                    #     print(index, 'one guy in dataset for this jersey', jersey, existing_data_names_found_this_jersey, list_of_names)
                    #     filtered_names_match_date = filter_using_date_of_match(list_of_names, row['Nationality'], date, False)[1]
                    #     print('filtered to', filtered_names_match_date)
                    # else:
                    #     print(index, 'more than one guy in dataset for this jersey', jersey, existing_data_names_found_this_jersey)
                    #     filtered_names_match_date = filter_using_date_of_match(list_of_names, row['Nationality'], date, False)[1]
                    #     print('filtered to', filtered_names_match_date)
                    #vetted_list_using_initials = vet_group_using_initials(jersey, list_of_names[0], list_of_names)
                    # if player_this in vetted_list_using_initials:
                    #     vetted_list_using_initials.remove(player_this)
                    print(index, list_of_names, jersey)
    else:
        done_cases += 1


#the match date shit isnt working. i think what u need to do is look for existing matches. if they dont have any you might just have to look it up.

#a lot of these guys are cases where the last name of the other person is a fucking sub. ugh 

done_cases, correct_cases, not_working_cases

2 ['Charalampos Kyriakou', 'Vangelis Kyriakou'] Cham Kyriakou
3 ['Daniel James', 'Jordan James'] James
4 ['Daniel James', 'Jordan James'] James
5 ['Daniel James', 'Jordan James'] James
6 ['Ricardo Silva', 'Ricardo Gomes'] Ricardo
7 ['Martin Büchel', 'Marcel Büchel'] M Büchel
8 ['Danel Sinani', 'Dejvid Sinani'] Sinani
9 ['Kyle Casciaro', 'Lee Casciaro'] Casciaro
10 ['Martin Büchel', 'Marcel Büchel'] M Büchel
13 ['Ben Davies', 'Adam Davies'] Davies
14 ['Kyle Casciaro', 'Lee Casciaro'] Casciaro
15 ['Javier Orozco', 'Jonathan Orozco'] J Orozco
16 ['Javier Orozco', 'Jonathan Orozco'] J Orozco
17 ['Javier Orozco', 'Jonathan Orozco'] J Orozco
19 ['Tonia Tisdell', 'Terrence Tisdell'] T Tisdell
20 ['Edson Álvarez', 'Efraín Álvarez'] E Álvarez
21 ['Edson Álvarez', 'Efraín Álvarez'] E Álvarez
22 ['Luuk de Jong', 'Frenkie de Jong'] de Jong
23 ['Luuk de Jong', 'Frenkie de Jong'] de Jong
24 ['Luuk de Jong', 'Frenkie de Jong'] de Jong
25 ['Luuk de Jong', 'Frenkie de Jong'] de Jong
26 ['Luuk de Jong',

(17, 0, 94)

In [136]:
name_confusion.to_csv('name_confusion_edited.csv')

In [125]:
# for index, row in combined_df.iterrows():
#     if combined_df.at[index, 'Name(s) Found'].startswith('['):
#         edited_name = eval(combined_df.at[index, 'Name(s) Found'])[0]
#         combined_df.at[index, 'Name(s) Found'] = edited_name
#         #print(index, combined_df.at[index, 'Name(s) Found'])

combined_df[combined_df['Name(s) Found'].str.startswith('[')]

,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case


## other_cases_wrong

In [227]:
other_cases = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Red Groups/other_cases_wrong_edited.csv')

In [215]:
# for index, row in other_cases.iterrows():
#     if row['Nationality'] == 'Brunei':
#         other_cases.at[index, 'Nationality'] = 'Brunei Darussalam'
#     if row['Nationality'] == 'China PR':
#         other_cases.at[index, 'Nationality'] = 'China'
#     if row['Nationality'] == 'French Guyana':
#         other_cases.at[index, 'Nationality'] = 'French Guiana'
#     if row['Nationality'] == 'Congo DR':
#         other_cases.at[index, 'Nationality'] = 'Democratic Republic of the Congo'

In [247]:
cases = 0
for index, row in other_cases.iterrows():
    if row['Nationality'] in ['Brunei Darussalam', 'China', 'French Guiana', 'Democratic Republic of the Congo']:
        if row['Status New'] != 'Done':
            match = row['Match']
            date = row['Date']
            jersey = row['ORIGINAL JERSEY']
            nationality = row['Nationality']

            full_num = row['Season']
            country_code_this = row['Country Code']
            dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique() 
            if len(dataset_nationality) <= 1:
                0==0
            else:    
                best_match = threshold_player_match(jersey, dataset_nationality)
                if best_match[1] >= 80:
                    vetted_group = vet_group_using_initials(jersey, jersey, best_match[0])
                    if jersey in vetted_group:
                        vetted_group.remove(jersey)
                        
                    if len(vetted_group) == 1:
                        print(index, full_num, jersey, country_code_this, vetted_group)
                        cases += 1
                        correct_name = vetted_group[0]

                        number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                        print(index, full_num, number, reason)
                        #fix the function 
    
    #mark the guys in the other list of nations as FAIL - NO DATA
    #['Martinique', 'French Guiana', 'Eritrea']
    #set status to FAIL - no data
    #
cases

11 2023 H Nyaring BN ['Haimie Nyaring']
0 0 0
11 2023 0 Did Not Find in Curr Season, One Up, or One Down
12 2023 Hanif Hamir BN ['Hanif Hamir']
0 0 0
12 2023 0 Did Not Find in Curr Season, One Up, or One Down
13 2023 Nazhan Zulki BN ['Nazhan Zulkifle']
0 0 0
13 2023 0 Did Not Find in Curr Season, One Up, or One Down
15 2023 Haziq Hasimulabdillah BN ['Haziq Kasyful']
0 0 0
15 2023 0 Did Not Find in Curr Season, One Up, or One Down
16 2023 Nur Asyraffahmi Norsamri BN ['Nur Asyraffahmi Norsamri']
0 0 0
16 2023 0 Did Not Find in Curr Season, One Up, or One Down
17 2023 H Yazid BN ['Hakeme Yazid Said']
0 0 0
17 2023 0 Did Not Find in Curr Season, One Up, or One Down
18 2023 Khairil Shahme Suhaimi BN ['Khairil Shahme Suhaimi']
0 0 0
18 2023 0 Did Not Find in Curr Season, One Up, or One Down
19 2023 Y Indera Putera BN ['Yura Indera Putera']
0 0 0
19 2023 0 Did Not Find in Curr Season, One Up, or One Down


20 2023 H Azam BN ['Hendra Azam']
0 0 0
20 2023 0 Did Not Find in Curr Season, One Up, or One Down
21 2023 Azizi Ali Rahman BN ['Azizi Ali Rahman']
0 0 0
21 2023 0 Did Not Find in Curr Season, One Up, or One Down
157 2023 H Nyaring BN ['Haimie Nyaring']
0 0 0
157 2023 0 Did Not Find in Curr Season, One Up, or One Down
158 2023 Nazhan Zulki BN ['Nazhan Zulkifle']
0 0 0
158 2023 0 Did Not Find in Curr Season, One Up, or One Down
159 2023 Y Indera Putera BN ['Yura Indera Putera']
0 0 0
159 2023 0 Did Not Find in Curr Season, One Up, or One Down
160 2023 Hanif Hamir BN ['Hanif Hamir']
0 0 0
160 2023 0 Did Not Find in Curr Season, One Up, or One Down
161 2023 A Mu'iz BN ["Abdul Mu'iz Sisa"]
0 0 0
161 2023 0 Did Not Find in Curr Season, One Up, or One Down
162 2023 H Yazid BN ['Hakeme Yazid Said']
0 0 0
162 2023 0 Did Not Find in Curr Season, One Up, or One Down
163 2023 Khairil Shahme Suhaimi BN ['Khairil Shahme Suhaimi']
0 0 0
163 2023 0 Did Not Find in Curr Season, One Up, or One Down
164

KeyboardInterrupt: 

In [230]:
other_cases['Nationality'].value_counts().head(30)

Nationality
Martinique                          122
Brunei Darussalam                    66
China                                56
Guadeloupe                           43
French Guiana                        33
Eritrea                              31
Cameroon                             28
Madagascar                           23
Israel                               23
Libya                                21
Faroe Islands                        19
Chinese Taipei                       18
Cyprus                               17
Sudan                                16
Hong Kong                            16
Togo                                 16
Niger                                15
Thailand                             15
Vietnam                              15
Belarus                              14
Democratic Republic of the Congo     14
Ethiopia                             14
Turkey                               14
Azerbaijan                           13
Yemen                       

In [222]:
other_cases[other_cases['Nationality'] == namey]['ORIGINAL JERSEY'].value_counts()

ORIGINAL JERSEY
R Yusef    11
H Gerba     2
D Tura      1
Name: count, dtype: int64

In [221]:
namey = 'Ethiopia'
leagues_value[leagues_value['Nationality'] == namey]['Season'].value_counts()

Season
2021    64
2019    53
2015    42
2017    42
2022    37
2016    34
2018    34
2011    33
2014    32
2020    28
2012    27
2023    23
2013    19
Name: count, dtype: int64

In [208]:
leagues_value_large[leagues_value_large["Nationality"] == namey]['Season'].value_counts()

Season
2021    64
2019    53
2015    42
2017    42
2022    37
2016    34
2018    34
2014    32
2020    28
2023    23
2013    19
Name: count, dtype: int64

In [226]:
other_cases.to_csv('other_cases_wrong_edited.csv')

## failed_finding_season_up_down

In [97]:
failed_all_seasons = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Red Groups/failed_finding_season_up_down.csv')

In [98]:
failed_all_seasons[failed_all_seasons['Match'] == 'England vs Iran']

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Match ID,Competition,Date,Match,Season,Nationality,Country Code,ORIGINAL JERSEY,Lookup Still Required?,Lookup Return Case,Name(s) Found,Translated Name,Original Case,To Remove
73,73,149,5,310,World Cup 2022,21.11.22,England vs Iran,2022,Iran,IR,Karimi,True,Failed Using Name Looking at Season and Season...,Ali Karimi,NaN,Group 1 Fail,False
